In [30]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from queue import Queue

class Amazon:
    def __init__(self, search, limit, page_visit): # , count_product
        self.search = search
        self.limit = limit
        # self.count_product = count_product
        self.result_queue = Queue()
        self.page_visit = page_visit
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.s)
        self.driver.get('https://www.amazon.in/')
        self.un = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="twotabsearchtextbox"]')))
        self.un.send_keys(search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)

    def link_collection(self):
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        first_page_id = self.driver.current_window_handle
        review = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,
                                                                                           'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))  # a-size-base.s-underline-text
        reviews = []
        price = []

        for i in review:
            if '₹' not in i.text:
                reviews.append(i)

        for i in review:
            if '₹' in i.text:
                price.append(i.text)
        # price = []

        cnt = 0
        for i in price:
            if cnt % 2 != 0:
                price.append(i)
            cnt += 1

        try:
            title1 = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-size-base-plus.a-color-base.a-text-normal')))
        except TimeoutException:
            title1 = []

        try:
            title2 = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-size-medium.a-color-base.a-text-normal')))
        except TimeoutException:
            title2 = []

        main_title = title1 if len(title1) > len(title2) else title2

        titles = []
        d = {}
        prices = {}
        for i, j in zip(main_title, reviews):
            x = self.search.lower()
            if ('(Renewed)' not in i.text) or ('WeConnect' not in i.text) or (
                    'Protect+' not in i.text):  # or ('AppleCare' not in i.text)
                d[j] = i.text
                titles.append(i.text)

        for i, j in zip(main_title, price):
            prices[i.text] = j[1:]

        time.sleep(1)
        return d, titles, prices

    def scraping(self, d, limit):  # , count_product
        # j =
        no_of_product = 0
        prod_review = {}
        ratings = {}
        link_of_product = {}
        overall_rating = {}
        for i in d:
            # if no_of_product == self.count_product:
            #     break
            y = d[i]
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            # driver.switch_to.window(first_page_id)
            i.click()
            time.sleep(2)
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)

            link = self.driver.current_url
            link_of_product[y] = link

            time.sleep(6)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')

            try:
                all_comments = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'a-link-emphasis.a-text-bold')))
                all_comments.click()
                time.sleep(2)
            except TimeoutException:
                pass

            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            try:
                box = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-dropdown-prompt')))
                for k in box:
                    if k.text == 'All formats':
                        k.click()
            except TimeoutException:
                pass

            try:
                click = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'format-type-dropdown_1')))
                click.click()
                time.sleep(3)
            except TimeoutException:
                pass

            # scrapping views
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            # scraping rating
            h = x.find_all('span', {'class': 'a-size-medium a-color-base'})

            clean = ' '
            for g in h:
                cleaned = g.text.strip()
                if g.text.strip() != '':
                    clean += cleaned[:3]
            overall_rating[y] = cleaned.strip()

            ## scraping review

            t = x.find_all('span', {'class': 'a-size-base review-text review-text-content'})
            reviews = set()
            for l in t:
                reviews.add(l.text.strip())
                if len(reviews) == self.limit:
                    break

            # star ratings
            h = x.find_all('td', {'class': 'a-text-right a-nowrap'})
            rating = {}
            counter = 5
            for rat in h:
                rating['{} Stars'.format(counter)] = rat.text.strip()
                counter -= 1

            ratings[y] = rating

            if len(reviews) != self.limit or len(reviews) < self.limit:
                try:
                    n = 0
                    while len(reviews) != self.limit:
                        try:
                            last = WebDriverWait(self.driver, 5).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-disabled.a-last')))
                            x = last.text
                            if x == 'Next page':
                                break
                        except (TimeoutException, StaleElementReferenceException):
                            review1 = reviews
                            next_ = WebDriverWait(self.driver, 10).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-last')))
                            next_.click()
                            time.sleep(4)
                            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                            e = self.driver.page_source
                            x = BeautifulSoup(e, 'html.parser')
                            t = x.find_all('div', {'class': 'a-size-base review-text review-text-content'})
                            for l in t:
                                reviews.add(l.text.strip())
                                if len(reviews) == self.limit:
                                    break
                            prod_review[y] = reviews
                            if len(reviews) == len(review1):
                                break
                except TimeoutException:
                    pass
            else:
                prod_review[y] = reviews

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])
            no_of_product += 1

        return prod_review, ratings, link_of_product, overall_rating

    def run(self, page_visit):
        review_of_product = []
        ratings_of_product = []
        link_product = []
        price_product = []
        overall_r = []

        for i in range(self.page_visit):
            d, titles, prices = self.link_collection()
            print(titles)
            prod, ratings, links, overall = self.scraping(d, self.limit) # , self.count_product

            review_of_product.append(prod)
            ratings_of_product.append(ratings)
            link_product.append(links)
            price_product.append(prices)
            overall_r.append(overall)

            self.driver.switch_to.window(self.driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(2)
            flag = False
            while flag != True:
                try:
                    next_ = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,
                                                                                                's-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')))
                    next_.click()
                    flag = True
                except (TimeoutException, ElementNotInteractableException, ElementClickInterceptedException):
                    self.driver.refresh()
                    time.sleep(3)
                    pass

        self.result_queue.put((review_of_product, price_product, overall_r, ratings_of_product, link_product))
        # return review_of_product, ratings_of_product, link_product, price_product, overall_r



In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from queue import Queue


class Croma:
    def __init__(self, search, limit, page_visit):  # , count_product
        self.search = search
        self.limit = limit
        self.page_visit = page_visit
        # self.count_product = count_product
        self.result_queue = Queue()
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.s)
        self.driver.get('https://www.croma.com/')
        self.flag = False

        while self.flag != True:
            try:
                # self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
                self.un = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="searchV2"]')))

                self.un.send_keys(self.search)
                self.flag = True
            except (ElementNotInteractableException, TimeoutException):
                self.driver.refresh()
                time.sleep(3)
                pass
        #                 self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
        #                 self.un.send_keys(self.search)
        #                 self.flag = True

        time.sleep(2)
        self.un.send_keys(Keys.ENTER)

    #         try:
    #             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
    #             #time.sleep(2)
    #             self.un.send_keys(search)
    #             time.sleep(2)
    #             self.un.send_keys(Keys.ENTER)
    #         except ElementNotInteractableException:
    #             self.driver.refresh()
    #             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
    #             #time.sleep(2)
    #             self.un.send_keys(search)
    #             time.sleep(2)
    #             self.un.send_keys(Keys.ENTER)

    def link_collection(self):

        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)

        link_texts = []

        te = self.search.split(' ')
        # y = []
        # print(x)
        for j in te:
            k = 0
            for i in range(len(j)):
                modified_string = f"{j[:i]}{j[i].upper()}{j[i + 1:]}"
                link_texts.append(modified_string)
                k += 1
                if k == 2:
                    break

        link = []
        for i in link_texts:
            try:
                review_link = WebDriverWait(self.driver, 5).until(
                    EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, i)))
                if len(review_link) != 0:
                    for i in review_link:
                        link.append(i)
            except TimeoutException:
                pass
        # print(link)

        #         review_link = WebDriverWait(self.driver, 10).until(
        #             EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, self.search)))
        # name = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))

        d = {}
        name = []
        for i in link:
            name.append(i.text)
            # print(i.text)
        # print(name)
        for i, j in zip(link, name):
            # x = self.search.lower()
            # if x in j.lower() and '(Renewed)' not in j:
            d[i] = j

        time.sleep(1)
        # print(d)
        return d

    def scrapings(self, d, limit):  # , count_product
        j = 1
        prod_review = {}
        no_of_product = 0
        link = {}
        price = {}
        ratings = {}
        avg_rating = {}

        for i in d:
            # if no_of_product == self.count_product:
            #     break
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            # i.click()
            webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()
            while len(self.driver.window_handles) != 2:
                time.sleep(3)
                webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()

            # driver.execute_script("arguments[0].click();", i)
            time.sleep(2)
            # driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)

            y = d[i]
            # scraping link
            l = self.driver.current_url
            link[y] = l

            # scraping price
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            p = []
            h = x.find_all('span', {'class': 'amount'})

            if len(h) <= 0:
                price[y] = p
            else:
                p.append(h[0].text)
                price[y] = p

            # time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            # try:
            #    all_comments = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'btn-wrap.view-all-review-btn')))
            # all_comments[0].click()
            #   webdriver.ActionChains(driver).move_to_element(all_comments).click(all_comments).perform
            # except TimeoutException:
            #    pass

            # scrapping ratings
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            r = []
            rating = {'5 Stars': 0, '4 Stars': 0, '3 Stars': 0, '2 Stars': 0, '1 Stars': 0}
            l = h = x.find_all('div', {'class': 'barAndStar'})
            # h = x.find_all('p', {'class' : 'right-text'})
            # rating['4 Stars'] = l[0].text
            if len(l) <= 0:
                ratings[y] = rating
            else:
                count = 5
                for i in l:
                    rating['{} Stars'.format(count)] = i.text[6:]
                    count -= 1

                ratings[y] = rating

            # scraping  avg ratings
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            t = []
            h = x.find_all('div', {'class': 'totalRating'})

            if len(h) <= 0:
                avg_rating[y] = t
            else:
                t.append(h[0].text)
                avg_rating[y] = t

            # time.sleep(3)

            # scraping review
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class': 'desc'})
            reviews = set()
            # y = d[i]
            for k in h:
                reviews.add(k.text)
                print(k.text)
                if len(reviews) == self.limit:
                    break
            # driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            # curr_height = driver.execute_script('return document.body.scrollHeight;')
            flag = 0
            reviews1 = reviews
            while len(reviews) != self.limit:
                try:
                    # driver.execute_script('window.scrollBy(0, 2000);')
                    # time.sleep(2)
                    flag += 1
                    more = WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, 'View All Reviews')))
                    webdriver.ActionChains(self.driver).move_to_element(more).click(more).perform()
                    e = self.driver.page_source
                    x = BeautifulSoup(e, 'html.parser')
                    t = x.find_all('div', {'class': 'desc'})
                    # print(curr_height)
                    # curr_height += 100
                    for l in t:
                        reviews.add(l.text.strip())
                        if len(reviews) == self.limit:
                            break

                    self.driver.execute_script('window.scrollBy(5350, 100);')
                    time.sleep(3)
                    print(flag)
                    if flag == 11:
                        if len(reviews1) == len(reviews):
                            break

                    # driver.execute_script('window.scrollBy(0, 2000);')
                except TimeoutException:
                    flag += 3
                    if flag >= 11:
                        break
                    else:
                        pass

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])
            no_of_product += 1
            print(j)
            j += 1

        return prod_review, ratings, link, avg_rating, price

    def run(self, page_visit):
        links = []
        reviews = []
        all_ratings = []
        prices = []
        avg_r = []

        main_d = {}
        now = time.time()
        for i in range(self.page_visit):
            d = self.link_collection()
            print(len(d))
            new_d = {}
            for i, j in d.items():
                if i in main_d:
                    continue
                else:
                    new_d[i] = j
            for i, j in d.items():
                main_d[i] = j
            x, y, z, a, b = self.scrapings(new_d, self.limit)  # , self.count_product
            reviews.append(x)
            all_ratings.append(y)
            links.append(z)
            avg_r.append(a)
            prices.append(b)

            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(3)
            next_page = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'btn.btn-secondary.btn-viewmore')))
            webdriver.ActionChains(self.driver).move_to_element(next_page).click(next_page).perform()

        self.result_queue.put((reviews, prices, avg_r, all_ratings, links))

        # return reviews, all_ratings, links, avg_r, prices
# x = Croma('iPhone 14', 10)




In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from queue import Queue


class Flipkart:
    def __init__(self, search, limit, page_visit):  # , count_product
        self.search = search
        self.limit = limit
        self.page_visit = page_visit
        # self.count_product = count_product
        self.result_queue = Queue()
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.s)
        self.driver.get('https://www.flipkart.com/')

        try:
            x_button = WebDriverWait(self.driver, 17).until(
                EC.presence_of_element_located((By.CLASS_NAME, '_2KpZ6l._2doB4z')))
            x_button.click()

        except TimeoutException:
            pass
        self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')))
        self.un.send_keys(self.search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)

    def link_collection(self):

        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)

        try:
            review = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, '_1fQZEK')))
            title = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, '_4rR01T')))

        except TimeoutException:
            review = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, '_2rpwqI')))
            title = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 's1Q9rs')))

        titles = []
        d = {}
        reviews = []

        try:
            for i in review:
                if '(' in i.text or '(' not in i.text:
                    reviews.append(i)
        except StaleElementReferenceException:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()
        try:
            for i, j in zip(title, reviews):
                # x = search.lower()
                if '(Renewed)' not in i.text:
                    d[j] = i.text
                    titles.append(i.text)
        except StaleElementReferenceException:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()
        # print(d)
        if len(d) == 0:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()

        time.sleep(1)
        return d

    def scraping(self, d, limit):  # , count_product)
        j = 1
        no_of_product = 0
        prod_review = {}
        link = {}
        overall_rating = {}
        price = {}
        ratings = {}
        for i in d:
            # if no_of_product == self.count_product:
            #     break
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            i.click()
            time.sleep(2)
            # driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)

            y = d[i]

            # scraping link
            url = self.driver.current_url
            link[y] = url

            # scraping stars
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class': '_1uJVNT'})
            stars = {}
            count = 5
            for i in h:
                stars["{} Stars".format(count)] = i.text
                count -= 1

            ratings[y] = stars

            # scraping price   <div class="_30jeq3 _16Jk6d">₹69,999</div>
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class': '_30jeq3 _16Jk6d'})
            p = []
            for i in h:
                p.append(i.text)
            price[y] = p

            # average ratings
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class': '_2d4LTz'})
            r = []
            for i in h:
                r.append(i.text)
            overall_rating[y] = r

            # time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            try:
                all_comments_link = WebDriverWait(self.driver, 6).until(
                    EC.presence_of_element_located((By.CLASS_NAME, '_3UAT2v._16PBlm')))
                self.driver.execute_script("arguments[0].click();", all_comments_link)

            except TimeoutException:
                pass

            # time.sleep(3)

            # scraping review
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class': 't-ZTKy'})
            reviews = set()
            # y = d[i]
            for k in h:
                reviews.add(k.text)
                if len(reviews) == limit:
                    break

            if len(reviews) != self.limit or len(reviews) < self.limit:
                try:
                    while len(reviews) != self.limit:
                        review1 = reviews
                        next_page = WebDriverWait(self.driver, 5).until(
                            EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
                        if len(next_page) == 1:
                            next_page[0].click()
                        elif len(next_page) == 2:
                            next_page[1].click()
                        time.sleep(4)
                        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                        e = self.driver.page_source
                        x = BeautifulSoup(e, 'html.parser')
                        t = x.find_all('div', {'class': 't-ZTKy'})
                        for l in t:
                            reviews.add(l.text.strip())
                            if len(reviews) == limit:
                                break
                        prod_review[y] = reviews

                except TimeoutException:
                    pass
            else:
                prod_review[y] = reviews

            # prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])
            no_of_product += 1

            print(j)
            j += 1

        return prod_review, link, overall_rating, price, ratings

    def run(self, page_visit):
        review = []
        avg_rating = []
        prices = []
        links = []
        all_r = []
        for i in range(self.page_visit):
            d = self.link_collection()
            print(len(d))
            x, y, z, a, b = self.scraping(d, self.limit)  # , self.count_product
            review.append(x)
            links.append(y)
            avg_rating.append(z)
            prices.append(a)
            all_r.append(b)

            # driver.switch_to.window(driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            next_page = WebDriverWait(self.driver, 5).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
            try:
                if len(next_page) == 1:
                    next_page[0].click()
                elif len(next_page) == 2:
                    next_page[1].click()
            except ElementClickInterceptedException:
                self.driver.refresh()
                time.sleep(4)
                self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')

                flag = False
                while flag != True:
                    try:
                        next_page = WebDriverWait(self.driver, 10).until(
                            EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
                        if len(next_page) == 1:
                            next_page[0].click()
                        elif len(next_page) == 2:
                            next_page[1].click()
                        flag = True
                    except (ElementNotInteractableException, TimeoutException, ElementClickInterceptedException):
                        self.driver.refresh()
                        time.sleep(3)
                        pass

        self.result_queue.put((review, prices, avg_rating, all_r, links))

        # return review, avg_rating, prices, links, all_r

In [ ]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    # Submit the functions to the executor for execution
    future1 = executor.submit(function1)
    future2 = executor.submit(function2)
    future3 = executor.submit(function3)

    # Wait for all futures to complete
    concurrent.futures.wait([future1, future2, future3])

In [50]:
import re
import numpy as np
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd
import re
import numpy as np
import re
import pandas as pd
from textblob import TextBlob
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import numpy as np
import re
import pandas as pd
from textblob import TextBlob
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer




count = 0
length = 0


class Cleaning:
    def __init__(self, review, price, avg_rating, link):  # all_rating
        self.review = review
        self.price = price
        self.avg_rating = avg_rating
        # self.all_rating = all_rating
        self.link = link

        self.df = pd.DataFrame(
            {'Product Name': [], 'Review': [], 'Avg Rating': [], 'Price': [],
             'Link': []})  # '5 Stars': [], '4 Stars': [], '3 Stars': [], '2 Stars': [],'1 Stars': [],

    def clean(self, review, price, avg_rating, link):  # , all_rating
        count = 0
        length1 = 0
        counter = 0
        for i in self.review:
            length = 0
            for j in i:
                # length = 0
                for k, l in j.items():
                    # print(k)
                    for m in l:
                        m = np.asarray(m)

                        # print(counter)
                        counter += 1
                        self.df.loc[count] = [str(k),str(m),  # self.all_rating[length1][length][k]['5 Stars'],
                                              # self.all_rating[length1][length][k]['4 Stars'],
                                              # self.all_rating[length1][length][k]['3 Stars'],
                                              # self.all_rating[length1][length][k]['2 Stars'],
                                              # self.all_rating[length1][length][k]['1 Stars'],
                                              str(self.avg_rating[length1][length][k]),
                                              str(self.price[length1][length][k]), str(self.link[length1][length][k])]
                        count += 1
                length += 1
            length1 += 1

        # cleaning rating
        x = self.df['Avg Rating']
        for i, j in enumerate(x):
            if type(j) == list():
                j = j[0]
                y = re.sub(r'[^0-9.]+','', j)
                y = y+ '   '
                y = y[:3].strip()
                self.df.loc[i, 'Avg Rating'] = y
            else:
                if type(j) == str:
                    y = re.sub(r'[^0-9.]+','', j)
                    y = y+ '   '
                    y = y[:3].strip()
                    self.df.loc[i, 'Avg Rating'] = y


        # cleeaning price
        x = self.df['Price']
        index = 0
        for i in x:
            if type(i) == list:
                y = i[0]
                y = y.split('.')
                y = y[0]
                y = re.findall(r'\d', y)
                string = ' '
                for j in y:
                    string += j
                y = float(string)
                # print(y)
                self.df.loc[index, 'Price'] = y
            else:
                y = i
                if '\n' in y:
                    y = y.split('\n')
                    y = y[0]
                y = y.split('.')
                y = y[0]
                y = re.sub(r'[\n₹,]', '', y)
                y = re.findall(r'\d', y)
                string = ' '
                for j in y:
                    string += j
                if len(string) == 0 or len(string) == 1:
                    string = 0
                else:
                    y = float(string)
                    
                if type(y) == list and len(y) == 0:
                    y = 0.0
                print(type(y), index)
                self.df.loc[index, 'Price'] = y

            index += 1

        
        cleaned = []
        for i in self.df['Review']:
            i = str(i)
            print(i)
            x = re.sub(r"[^a-zA-Z\s.,!?']", "", i)
            y = re.sub(r'READ MORE', '', x)
            # y = re.sub(r'[^\x00-\x7F]+', '', x)
            cleaned.append(y)

        print(' Cleaning done---------')
        print('Tokenizing')
        token = []
        stop_words = set(stopwords.words('english'))
        for i in cleaned:
            word_tokens = word_tokenize(i)
            # print(word_tokens)
            filtered_words = [word for word in word_tokens if not word.lower() in stop_words]
            word = ' '.join(filtered_words)
            words = re.sub('[^a-zA-Z0-9 \n]', '', word)
            print(words)
            token.append(words)
        self.df['Cleaned'] = token
        print()
        print('Done ----------')
        return self.df

    # print('-------------')
    def predict(self, data):
        def get_sentiment(text):
            return TextBlob(text).sentiment.polarity

        self.df['sentiment'] = self.df['Cleaned'].apply(get_sentiment)
        sentiment = self.df['sentiment']
        avg = self.df.groupby('Product Name')['sentiment'].mean().sort_values(ascending=False)[:1]
        best_product = avg.index
        polarity = avg.values
        price = self.df.loc[self.df['Product Name'] == best_product[0], 'Price'].values[0]
        link = self.df.loc[self.df['Product Name'] == best_product[0], 'Link'].values[0]
        rate = self.df.loc[self.df['Product Name'] == best_product[0], 'Avg Rating'].values[0]
    
    def group(self, df):
        groupby_of_all = df.groupby('Product Name')['Cleaned'].apply(list)
        return groupby_of_all
    
   
    def aspect_based_sentiment(self, groupby_of_all):
        se = {}
        prods = groupby_of_all.index
        for i, j in enumerate(groupby_of_all.values):
            list = []
            for k in j:
                x = nltk.word_tokenize(k)
                y = nltk.pos_tag(x)
                list.append(y)
            se[prods[i]] = list

        products_review_with_sentiments = {}
        for i, j in se.items():
            d1 = {}
            for k in j:
                for l, m in enumerate(k):
                    if len(m[0]) > 4 and '...' not in m[0] and '....':
                        if m[1] == 'NN' or m[1] == 'VB':
                            aspect = m[0]
                            sentence = ''
                            if aspect not in d1.keys():
                                d1[aspect] = []
                            for n in k[l:]:
                                sentence += n[0] + ' '
                                if m[1] == 'JJ':
                                    break
                            x = d1[aspect]
                            x.append(sentence.strip())
                products_review_with_sentiments[i] = d1


        def sentiment(d):
            sid = SentimentIntensityAnalyzer()
            for i, j in d.items():
                #print(j)
                for k, l in j.items():
                    d2 = {}
                    for m in l:
                        sentiment = sid.polarity_scores(m)
                        d2[m] = sentiment

                    d[i][k] = d2
            return d

        x = sentiment(products_review_with_sentiments)


        # overall sentiment
        compound_aspect = {}
        for i in x.keys():
            asp = x[i]
            pos = 0
            neg = 0
            neu = 0 
            compound = 0
            count = 0
            for j in asp.keys():
                comment = x[i][j]
                for k in comment.values():
                    pos += k['pos']
                    neg += k['neg']
                    neu += k['neu']
                    compound += k['compound']
                    count+= 1
            try:
                compound_aspect[i] = compound/ count 
            except ZeroDivisionError:
                compound_aspect[i] = 0

        # each aspect overall sentiment
        pro_asp_info = {}
        for i in x.keys():
            asp = x[i]
            aspect_info = {}
            for j in asp.keys():
                pos = 0
                neg = 0
                neu = 0 
                compound = 0
                count = 0
                comment = x[i][j]
                #print(comment)
                #print(j)
                for k in comment.values():
                    pos += k['pos']
                    neg += k['neg']
                    neu += k['neu']
                    compound += k['compound']
                    count+= 1

                info = {'pos' : pos/ count, 'neg' : neg/ count, 'neu' : neu/ count, 'compound' : compound/ count}

                aspect_info[j] = info
            pro_asp_info[i] = aspect_info




        best_products = {}
        data = df.Price
        low = 1
        high = 10
        for i in range(10):

            if low == 1:
                lower_quartile = data.min()
            lower_quartile = np.percentile(data, low)
            high_quartile = np.percentile(data, high)

            # quartile
            closest_min_value = min(data, key=lambda x: abs(x - lower_quartile))
            closest_max_value = min(data, key=lambda x: abs(x - high_quartile))

            s1 = df.loc[(df.Price >= closest_min_value) & (df.Price <= closest_max_value)]

            pro = s1['Product Name'].unique()
            fore ={}
            for i in pro:
                x1 = compound_aspect[i]
                fore[i] = x1
                #print(i)
            maxe = max(fore.values())
            #print("Products between {} - {} Quartile".format(low, high))

            pro = [k for k, v in fore.items() if v == maxe]
            #print(pro)

            prods = {}
            prods[pro[0]] = maxe
            best_products['Products between {} - {} Quartile'.format(low, high)] = prods 



            #print('==============')
            #print()
            low += 10
            high +=10
            if high > 100:
                break
        price = {}
        link = {}
        rating = {}
        #print(best_products)
        for k, v in best_products.items():
            price1 = {}
            link1 = {}
            rat = {}
            print(v)
            #print(list(v.keys()))
            a = [k for k, v1 in v.items()][0]
    #             for i in v.keys():
    #                 print(i)
            x1 = self.df.loc[df['Product Name'] == a]
            p = x1.Price.unique()[0]
            links = x1.Link.unique()[0]
            ratings = x1['Avg Rating'].unique()[0]
            price1[a] = p
            link1[a] = links
            rat[a] = ratings

            rating[k] = rat
            price[k] = price1
            link[k] = link1




        return best_products, price, link, rating
    
        #return best_product, polarity, link, price


In [1]:
data1 = input()

earbuds under 2000


In [70]:
data2 = int(input())

15


In [3]:
data3 = int(input())

1


In [8]:
data4 = int(input())

4


In [71]:
x = Croma(data1, data2, data3)
y = Flipkart(data1, data2, data3)
z = Amazon(data1, data2, data3)

t3 = threading.Thread(target=z.run(data3))
t1 = threading.Thread(target=x.run(data3))
t2 = threading.Thread(target=y.run(data3))


t3.start()
t1.start()
t2.start()


t3.join()
t1.join()
t2.join()

# data collection
review = []
price = []
avg_rating = []
all_rating = []
link = []

for i in [x, y, z]:
    x, y, z, a, b = i.result_queue.get()
    review.append(x)
    price.append(y)
    avg_rating.append(z)
    all_rating.append(a)
    link.append(b)


# run
#review_of_product, ratings_of_product, link_product, price_product, overall_r  = amazon.run(data3)

# pass cleaning.
print(all_rating)



['truke Newly Launched BTG Neo Dual Pairing Earbuds with 6-Mic Advanced ENC, 80H Playtime, 35ms Ultra-Low Latency, 13mm Titanium Drivers, 3 EQ Modes, Fast Charge, Instant Pairing, Bluetooth 5.3, IPX5', 'Boult Audio Z40 True Wireless in Ear Earbuds with 60H Playtime, Zen™ ENC Mic, Low Latency Gaming, Type-C Fast Charging, Made in India, 10mm Rich Bass Drivers, IPX5, Bluetooth 5.1 Ear Buds TWS (Brown)', 'boAt Airdopes Atom 81 True Wireless Earbuds with Upto 50H Playtime, Quad Mics ENx™ Tech, 13MM Drivers,Super Low Latency(50ms), ASAP™ Charge, BT v5.3(Opal Black)', 'Oppo Enco Air2i Bluetooth Truly Wireless in-Ear Earbuds with Mic, Fast Charging & Up to 28Hrs Battery - White', 'TOZO A1 Mini Wireless\u2002Earbuds Bluetooth 5.3 in Ear Light-Weight Headphones Built-in Microphone, IPX5 Waterproof, Immersive Premium Sound Long Distance Connection Headset with Charging Case, Black', 'Ekko Earbeats T08 Bluetooth Truly Wireless in Ear Earbuds with mic, ENC Call Noise Cancellation, 40H Playtime,10M

1
2
3
20
40
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
[[{'pTron Bassbuds Air 140318669 TWS Earbuds with Passive Noise Cancellation (IPX4 Water Resistant, Touch Control, Black)': {'5 Stars': '3', '4 Stars': '1', '3 Stars': 0, '2 Stars': 0, '1 Stars': 0}, 'boAt Airdopes 148 TWS Earbuds (IPX4 Sweat & Water Resistant, 42 Hours Playtime, Cyan Cider)': {'5 Stars': '157', '4 Stars': '34', '3 Stars': '7', '2 Stars': '7', '1 Stars': '7'}, 'realme Techlife RMA2109 Earbuds with AI Environment Noise Cancellation (IPX5 Water Resistant, Google Fast Pair, Black)': {'5 Stars': '113', '4 Stars': '11', '3 Stars': '6', '2 Stars': '1', '1 Stars': '5'}, 'pTron Bassbuds Air 140318670 TWS Earbuds with Passive Noise Cancellation (IPX4 Water Resistant, Touch Control, Blue)': {'5 Stars': '1', '4 Stars': '3', '3 Stars': 0, '2 Stars': 0, '1 Stars': 0}, 'boAt Airdopes 131 Pro TWS Earbuds with Environmental Noise Cancellation Technology (IPX5 Wate

In [ ]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    # Submit the functions to the executor for execution
    future1 = executor.submit(x.run(data3))
    future2 = executor.submit(y.run(data3))
    future3 = executor.submit(z.run(data3))

    # Wait for all futures to complete
    concurrent.futures.wait([future1, future2, future3])

In [73]:
df

,Product Name,Review,Avg Rating,Price,Link,Cleaned
0,pTron Bassbuds Air 140318669 TWS Earbuds with ...,Received wrong product please replace my order\n,3.7,599.0,https://www.croma.com/ptron-bassbuds-air-14031...,Received wrong product please replace order
1,pTron Bassbuds Air 140318669 TWS Earbuds with ...,"According to the price, these earbuds are of v...",3.7,599.0,https://www.croma.com/ptron-bassbuds-air-14031...,According price earbuds good quality sound q...
2,boAt Airdopes 148 TWS Earbuds (IPX4 Sweat & Wa...,"Product is great, service is great",4.5,1299.0,https://www.croma.com/boat-airdopes-148-tws-ea...,Product great service great
3,boAt Airdopes 148 TWS Earbuds (IPX4 Sweat & Wa...,Voice quality is bad. Noise cancellation not a...,4.5,1299.0,https://www.croma.com/boat-airdopes-148-tws-ea...,Voice quality bad Noise cancellation available
4,boAt Airdopes 148 TWS Earbuds (IPX4 Sweat & Wa...,The product is Very good and all the facilitie...,4.5,1299.0,https://www.croma.com/boat-airdopes-148-tws-ea...,product good facilities communication skills b...
...,...,...,...,...,...,...
693,"EDYELL TWS Wireless Earbuds, True Gaming Mode,...",Excellent1,4.3,1999.0,https://www.amazon.in/EDYELL-Wireless-Playtime...,Excellent
694,"EDYELL TWS Wireless Earbuds, True Gaming Mode,...",Best Product The soud quality are awesome and ...,4.3,1999.0,https://www.amazon.in/EDYELL-Wireless-Playtime...,Best Product soud quality awesome gaming mod a...
695,"EDYELL TWS Wireless Earbuds, True Gaming Mode,...",Better bass and sound quality.Too good for gam...,4.3,1999.0,https://www.amazon.in/EDYELL-Wireless-Playtime...,Better bass sound qualityToo good gamingNo sou...
696,"EDYELL TWS Wireless Earbuds, True Gaming Mode,...",Excellent for gaming and for music lovers,4.3,1999.0,https://www.amazon.in/EDYELL-Wireless-Playtime...,Excellent gaming music lovers


In [74]:
df.to_csv('Earbuds1.csv')

In [72]:
x = Cleaning(review, price, avg_rating,  link) # all_rating,
df = x.clean(review, price, avg_rating,  link) # all_rating,
y = x.group(df)
bp, p, l, r = x.aspect_based_sentiment(y)
# predict
#products, polaritys, links, prices, rates = x.predict(df)

<class 'float'> 0
<class 'float'> 1
<class 'float'> 2
<class 'float'> 3
<class 'float'> 4
<class 'float'> 5
<class 'float'> 6
<class 'float'> 7
<class 'float'> 8
<class 'float'> 9
<class 'float'> 10
<class 'float'> 11
<class 'float'> 12
<class 'float'> 13
<class 'float'> 14
<class 'float'> 15
<class 'float'> 16
<class 'float'> 17
<class 'float'> 18
<class 'float'> 19
<class 'float'> 20
<class 'float'> 21
<class 'float'> 22
<class 'float'> 23
<class 'float'> 24
<class 'float'> 25
<class 'float'> 26
<class 'float'> 27
<class 'float'> 28
<class 'float'> 29
<class 'float'> 30
<class 'float'> 31
<class 'float'> 32
<class 'float'> 33
<class 'float'> 34
<class 'float'> 35
<class 'float'> 36
<class 'float'> 37
<class 'float'> 38
<class 'float'> 39
<class 'float'> 40
<class 'float'> 41
<class 'float'> 42
<class 'float'> 43
<class 'float'> 44
<class 'float'> 45
<class 'float'> 46
<class 'float'> 47
<class 'float'> 48
<class 'float'> 49
<class 'float'> 50
<class 'float'> 51
<class 'float'> 52
<cl

<class 'float'> 591
<class 'float'> 592
<class 'float'> 593
<class 'float'> 594
<class 'float'> 595
<class 'float'> 596
<class 'float'> 597
<class 'float'> 598
<class 'float'> 599
<class 'float'> 600
<class 'float'> 601
<class 'float'> 602
<class 'float'> 603
<class 'float'> 604
<class 'float'> 605
<class 'float'> 606
<class 'float'> 607
<class 'float'> 608
<class 'float'> 609
<class 'float'> 610
<class 'float'> 611
<class 'float'> 612
<class 'float'> 613
<class 'float'> 614
<class 'float'> 615
<class 'float'> 616
<class 'float'> 617
<class 'float'> 618
<class 'float'> 619
<class 'float'> 620
<class 'float'> 621
<class 'float'> 622
<class 'float'> 623
<class 'float'> 624
<class 'float'> 625
<class 'float'> 626
<class 'float'> 627
<class 'float'> 628
<class 'float'> 629
<class 'float'> 630
<class 'float'> 631
<class 'float'> 632
<class 'float'> 633
<class 'float'> 634
<class 'float'> 635
<class 'float'> 636
<class 'float'> 637
<class 'float'> 638
<class 'float'> 639
<class 'float'> 640


using days writing review  Sound okay nothing great  Noise cancellation good atall person hear lots disturbance call  Battery long lasting used total days one complete charge use daily atleast hours official calls  Secondly takes less hour charge  Boat work sound quality bluetooth devices wired headphones par quality 
First thanks Flipkart Boat  discuss headphone  Sound Quality Bass unique sound quality  use lots headphone unique sweet sound quality  Bass enough listening  Volume upto enough listen  give star sound star bass  Quality Quality bad  star quality  Calling issues calling  awesome calling quality  Battery s great 
Rockerz pro excellent value money product BoatBattery backup also outstanding  build  sound quality performance also mind blowingI really happy neckband also comfortable ears 
neckband looks good price especially battery backup fast charging  Sound good bass could better  use check better opinion  Call quality good though  Cons nt fit well ears right side feels lit

media could loaded  IntroductionOver past days  pleasure using extraordinary Bluetooth earbud truly transformed audio experience  impressive bass response lightningfast charging ultralow latency  earbud exceeded expectations every aspect  comprehensive review  delve key features make earbud standout performer explain believe absolute gamechangerBass PerformanceOne standout features Bluetooth earbud exceptional bass performance  audio quality simply remarkable  delivering deep  rich bass brings music life  Whether m listening favorite tracks immersing movie  earbud reproduces low frequencies remarkable accuracy depth  adds whole new dimension audio experience  allowing feel music truly appreciate intricacies beatFast ChargingThe fast charging capability earbud nothing short impressive  advanced charging technology  fully charge earbud fraction time compared wireless earbuds ve used past  convenience able quickly charge earbud gamechanger  especially m go rush  ensures never worry runnin

{'Aroma NB140 Fabulous 24 Hours* Playtime | Deep Bass | M...': 0.7275275362318842}
{'truke Newly Launched BTG Storm Gaming Earbuds with 40ms Ultra-Low Latency, 13mm Titanium Drivers, ENC, 48H Playtime, Fast Charge, BT 5.3, IPX5 (Black)': 0.7712910958904111}
{'truke Newly Launched BTG Storm Gaming Earbuds with 40ms Ultra-Low Latency, 13mm Titanium Drivers, ENC, 48H Playtime, Fast Charge, BT 5.3, IPX5 (Black)': 0.7712910958904111}
{'boAt Rockerz 510 Super Extra Bass Bluetooth Headset': 0.6582846153846151}
{'boAt Rockerz 255 Pro+ /258 Pro+ with ASAP Charge and up...': 0.5190201388888888}
{'Boult Audio Z25 True Wireless in Ear Earbuds with 32H Playtime, 45ms Low Latency, Type-C Fast Charging (10=150Mins), Made in India, Zen ENC Mic, 13mm Bass Drivers, Bluetooth 5.3 Ear Buds (Sunset Grey)': 0.5500117647058826}
{'Boult Audio AirBass Z40 TWS Earbuds with Environmental Noise Cancellation (IPX5 Water Resistant, Voice Assistant, Blue)': 0.7534818181818181}
{'Boult Audio AirBass Z40 TWS Earbuds w

In [75]:
prr  = [k for k, v in p['Products between 31 - 40 Quartile'].items()]

In [76]:
df.loc[df['Product Name'] == prr[0]]['Link']

410    https://www.flipkart.com/boat-rockerz-510-supe...
411    https://www.flipkart.com/boat-rockerz-510-supe...
412    https://www.flipkart.com/boat-rockerz-510-supe...
413    https://www.flipkart.com/boat-rockerz-510-supe...
414    https://www.flipkart.com/boat-rockerz-510-supe...
415    https://www.flipkart.com/boat-rockerz-510-supe...
416    https://www.flipkart.com/boat-rockerz-510-supe...
417    https://www.flipkart.com/boat-rockerz-510-supe...
418    https://www.flipkart.com/boat-rockerz-510-supe...
419    https://www.flipkart.com/boat-rockerz-510-supe...
420    https://www.flipkart.com/boat-rockerz-510-supe...
421    https://www.flipkart.com/boat-rockerz-510-supe...
422    https://www.flipkart.com/boat-rockerz-510-supe...
423    https://www.flipkart.com/boat-rockerz-510-supe...
424    https://www.flipkart.com/boat-rockerz-510-supe...
Name: Link, dtype: object

In [79]:
x = df['Avg Rating']
for i, j in enumerate(x):
    j = str(j)
    y = re.sub(r'[a-zA-z]','', j)
    y = y.split(' ')[0]
    y = float(y)
    df.loc[i, 'Avg Rating'] = y

In [80]:
df

,Product Name,Review,Avg Rating,Price,Link,Cleaned,sentiment
0,"Apple iPhone 14 (128GB, Blue)",Its great and thanks to the customer care guy ...,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,great thanks customer care guy Mr Seikh helped...,0.428571
1,"Apple iPhone 14 (128GB, Blue)",It was good,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,good,0.700000
2,"Apple iPhone 14 (128GB, Blue)",Great product,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,Great product,0.800000
3,"Apple iPhone 14 (128GB, Blue)",Got the best deal,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,Got best deal,1.000000
4,"Apple iPhone 14 (128GB, Starlight)",Amazing experience,4.8,71990.0,https://www.croma.com/apple-iphone-14-128gb-st...,Amazing experience,0.600000
5,"Apple iPhone 14 (128GB, Starlight)",Product was amazing,4.8,71990.0,https://www.croma.com/apple-iphone-14-128gb-st...,Product amazing,0.600000
6,"Apple iPhone 14 (128GB, Starlight)",Nice product,4.8,71990.0,https://www.croma.com/apple-iphone-14-128gb-st...,Nice product,0.600000
7,"Apple iPhone 14 (128GB, Starlight)",Excellent,4.8,71990.0,https://www.croma.com/apple-iphone-14-128gb-st...,Excellent,1.000000
8,"Apple iPhone 14 (128GB, Purple)",Fantastic,4.8,71990.0,https://www.croma.com/apple-iphone-14-128gb-pu...,Fantastic,0.400000
9,"Apple iPhone 14 (128GB, Purple)",Awesome product,4.8,71990.0,https://www.croma.com/apple-iphone-14-128gb-pu...,Awesome product,1.000000


In [110]:
x =  avg_rating[0][0]['Apple iPhone 14 (128GB, Blue)']
x = "['4.6'] out of 5"

y = re.sub(r'[^0-9.]+','', x)
y+= '   '
y[:3].strip()

'4.6'